Note that it might make more sense to just request cov(x,y), std(x), std(y) and compute correlation on requester's end instead of requesting the whole HUGE circuit of correlation. 

In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

This example is not necessary. In fact, a person can just request cov(x,y), std(x), and std(y). Then just compute correlation on his own as well, but here we show that the code is composable enough to do all at once. 

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [14]:
scales = [5]
selected_columns = ['x', 'y']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [15]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x, y = data[0], data[1]
    return s.correlation(x, y)

error = 0.01
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

In [16]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.


==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 5, param_scale: 5, scale_input_multiplier: 1) ------------->

+------------+--------------+------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error | median_error | max_error  | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+------------+--------------+------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0.00537169 | 0.01074338   | 0.01074338 | 0         | 0.00537169     | 0.01074338       | 0.01074338    | 0             | 0.000057710102     | 0.0073635895       | 0.0073635895           |
+------------+--------------+------------+-----------+----------------+------------------+---------------+--------------

scale:  [5]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":5,"param_scale":5,"scale_rebase_multiplier":1,"lookup_range":[-32360,32536],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":7872,"total_assignments":3710,"total_const_size":792,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,5],"model_input_scales":[5,5],"module_sizes":{"kzg":[],"poseidon":[7872,[2]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"required_range_checks":[[-16,16]],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709724514721}


In [17]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 6.577926874160767 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 0.71875
==== Generating Proof ====
proof:  {'instances': [['7469705c7b45875730aac31b009e8f9e9f9e57031bba148110877f78e4518c22', 'c1f946e3fd2b2229bfcc2dc9ff7f4dfc1fddad53635becb58d130bcae5504712', '0100000000000000000000000000000000000000000000000000000000000000', '1700000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x1a09879e14b606ba8c989eb87435765f15c8ec5fb39e8ae1641c18657545a4da1feae65c8e87459f1c446f95db285227254017d6bd4a130eac619598e2a932ae247278c145339cedb5acdd8c6b82cd5544d1eee7de2feca5deba7079bee562be0eec74a4cc7d1276fe29976bd39ced477edcc57650929e1a0868cf4dce4c0d480a928ac0f4a5659dc1abbe6afccf70e2f41662045c75de5d0fe473731415e61c21de2ade6213ea5e414386b75105c0029fa67a24d825caced2ac3c0c3d8425fd08cafafe02290541e5951dfd8db80872d0464dded84f2a8b00f27060d0ef0afd18985f1e05e606083a9b65ab77df38831d595937a55c9cc4ff843c09380ae1a618db3b6fd

In [18]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: [0.71875]
